In [55]:
#!pip install -q wot

In [56]:
import ipywidgets as widgets
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import wot

Load transport map model and cell sets

In [57]:
tmap_model = wot.tmap.TransportMapModel.from_directory('tmaps/serum')
cell_sets = wot.io.read_sets('data/cell_sets.gmt', as_dict=True)

Convert cell sets indicating whether a cell at time t is in a cell set

In [58]:
populations = tmap_model.population_from_cell_sets(cell_sets, at_time=18)

Print cell set name and size

In [59]:
for pop in populations:
    print('{}, {}'.format(pop.name, np.count_nonzero(pop.p)))

IPS, 412
Stromal, 1239
Neural, 819
Epithelial, 503
Epithelial_clust, 1013
Trophoblast, 292
OPC, 141
Astrocyte, 112
Neuron, 122
RG_Gdf10, 144
RG_Id3, 160
RG_Neurog2, 21
RG, 255
OtherNeural, 390
TrophoLarger, 1305


Compute trajectories. Trajectory dataset has cells on rows and cell set populations on columns

In [60]:
trajectory_ds = tmap_model.compute_trajectories(populations)

Load embedding coordinates

In [61]:
coord_df = pd.read_csv('data/fle_coords.txt', sep='\t', index_col=0)
nbins = 500
xrange = coord_df['x'].min(), coord_df['x'].max()
yrange = coord_df['y'].min(), coord_df['y'].max()
coord_df['x'] = np.floor(
    np.interp(coord_df['x'], [xrange[0], xrange[1]], [0, nbins - 1])).astype(int)
coord_df['y'] = np.floor(
    np.interp(coord_df['y'], [yrange[0], yrange[1]], [0, nbins - 1])).astype(int)
trajectory_ds.obs = trajectory_ds.obs.join(coord_df)

Visualize trajectories

In [62]:
trajectory_dropdown = widgets.Dropdown(
    options=trajectory_ds.var.index,
    description='Trajectory:'
)

def update_trajectory_vis(name):
    figure = plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.tight_layout()
    plt.title(name)
    plt.scatter(coord_df['x'], coord_df['y'], c='#f0f0f0',
                   s=4, marker=',', edgecolors='none', alpha=0.8)
    binned_df = trajectory_ds.obs.copy()
    binned_df['values'] = trajectory_ds[:, name].X
    binned_df = binned_df.groupby(['x', 'y'], as_index=False).sum()
    plt.scatter(binned_df['x'], binned_df['y'], c=binned_df['values'],
                   s=6, marker=',', edgecolors='none', vmax=0.01)
    plt.colorbar().ax.set_title('Trajectory')

widgets.interact(update_trajectory_vis, name=trajectory_dropdown)

interactive(children=(Dropdown(description='Trajectory:', options=('IPS', 'Stromal', 'Neural', 'Epithelial', '…

<function __main__.update_trajectory_vis(name)>

Compute cell fates

In [63]:
fate_ds = tmap_model.compute_fates(populations)

Visualize cell fates

In [64]:
fate_dropdown1 = widgets.Dropdown(
    options=fate_ds.var.index,
    description='Fate 1:'
)
fate_dropdown2 = widgets.Dropdown(
    options=fate_ds.var.index,
    description='Fate 2:',
    value=fate_ds.var.index[1]
)

def update_fate_vis(name1, name2):
    figure = plt.figure(figsize=(10, 10))
    fate1 = fate_ds[:, name1].X
    fate2 = fate_ds[:, name2].X
    p = np.log(1e-9 + np.divide(fate1, fate2, out=np.zeros_like(fate1), where=fate2 != 0))
    plt.scatter(fate_ds.obs['day'], p, s=4, marker=',')
    plt.xlabel('Day')
    plt.ylabel('Log Odds')
    plt.title('{} vs. {}'.format(name1, name2))

widgets.interact(update_fate_vis, name1=fate_dropdown1, name2=fate_dropdown2)

interactive(children=(Dropdown(description='Fate 1:', options=('IPS', 'Stromal', 'Neural', 'Epithelial', 'Epit…

<function __main__.update_fate_vis(name1, name2)>